# Get Text data

In [1]:
import urllib.request
url = ("https://raw.githubusercontent.com/rasbt/LLMs-from-scratch/main/ch02/01_main-chapter-code/the-verdict.txt")
file_path = "the-verdict2.txt"
urllib.request.urlretrieve(url, file_path)

('the-verdict2.txt', <http.client.HTTPMessage at 0x7fc6dc225810>)

In [2]:
with open(file_path, "r", encoding="utf-8") as f:
    raw_text = f.read()

print("Total num char", len(raw_text))
print(raw_text[:99])

Total num char 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


# Bring in tokenizer

In [4]:
import tiktoken
tokenizer = tiktoken.get_encoding("gpt2")

In [7]:
# test tokenizer
tokens = tokenizer.encode("I love you")
print(tokens)

[40, 1842, 345]


In [8]:
print(tokenizer.decode(tokens))

I love you


In [13]:
# A wrapper of pytorch dataloader class
# converts raw_text into datasets
# creates input-output pairs

import torch
from torch.utils.data import Dataset, DataLoader

class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []
        token_ids = tokenizer.encode(txt)

        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))
    def __len__(self):
        return len(self.input_ids)
    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

In [14]:
# test the dataloader class 
# converts raw_text into datasets

max_length = 4
stride = 1

dataloader = GPTDatasetV1(raw_text, tokenizer, max_length, stride)
dataloader[0]


(tensor([  40,  367, 2885, 1464]), tensor([ 367, 2885, 1464, 1807]))

In [15]:
# dataloader in batches
def create_dataloader_v1(txt, batch_size=4, max_length=256,
    stride=128, shuffle=True, drop_last=True,
    num_workers=0):
    tokenizer = tiktoken.get_encoding("gpt2")
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)
    dataloader = DataLoader(
                    dataset,
                    batch_size=batch_size,
                    shuffle=shuffle,
                    drop_last=drop_last,
                    num_workers=num_workers
                )
    return dataloader

In [18]:
max_length = 4
dataloader = create_dataloader_v1(
    raw_text, batch_size=8, max_length=max_length,
    stride=max_length,shuffle=False
)

data_iter = iter(dataloader)
inputs, outputs = next(data_iter)
print("inputs", inputs.shape)
print("outputs", outputs.shape)

inputs torch.Size([8, 4])
outputs torch.Size([8, 4])


In [17]:
vocab_size = 50257
output_dim = 256
token_embedding_layer = torch.nn.Embedding(vocab_size,output_dim)

In [19]:
token_embeddings = token_embedding_layer(inputs)
token_embeddings.shape

torch.Size([8, 4, 256])

In [20]:
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)
pos_embedding = pos_embedding_layer(torch.arange(context_length))
pos_embedding.shape

torch.Size([4, 256])

In [22]:
input_embeddings = token_embeddings + pos_embedding
input_embeddings.shape

torch.Size([8, 4, 256])